In [1]:
import glob
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import xarray as xr
import multiprocessing as mp
import matplotlib.pyplot as plt
import copy
import matplotlib.colors as mcolors

import matplotlib.lines as mlines
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl
mpl.rcParams['hatch.linewidth'] = 0.5

# Load WRs

In [2]:
names_reanalyses = ['ERA5',
                   'JRA3Q',
                   'NCEP_NCAR']

dic_labels = {}
for reanalysis in names_reanalyses:
    labels_temp = pd.read_csv(f'../Data_v4/Labels/df_labels_{reanalysis}_v4.csv', 
                              parse_dates=True, index_col=0, names=['WR','distance'], skiprows=1)
    labels_temp['season'] = labels_temp.index.month % 12 // 3 + 1  # This will give: 1=DJF, 2=MAM, 3=JJA, 4=SON
    # Map season numbers to season names
    season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
    labels_temp['season'] = labels_temp['season'].map(season_map)
    dic_labels[reanalysis] = labels_temp

dic_events = {}

for ir, reanalysis in enumerate(names_reanalyses):
    # Compute Overall Frequency of Each Class
    df_labels = copy.deepcopy(dic_labels[reanalysis])
    df_labels['season'] = df_labels.index.month % 12 // 3 + 1  # This will give: 1=DJF, 2=MAM, 3=JJA, 4=SON
    # Map season numbers to season names
    season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
    df_labels['season'] = df_labels['season'].map(season_map)
    
    # Step 1: Identify changes in class to find the start of each event
    df_labels['shifted'] = df_labels['WR'].shift(1)
    df_labels['start'] = df_labels['WR'] != df_labels['shifted']
    df_labels['start_date'] = df_labels.index.where(df_labels['start'], pd.NaT)
    df_labels['start_date'].fillna(method='ffill', inplace=True)
    
    # Step 2: Calculate the duration of each event
    # Convert the Timedelta to its 'days' component
    df_labels['duration'] = (df_labels.index - df_labels['start_date']).dt.days + 1
    
    # Step 3: Create the df_events DataFrame
    # Group by 'start_date' and 'class' to get the duration of each class event
    df_events = df_labels.groupby(['start_date', 'WR']).agg({'duration': 'max'}).reset_index()
    
    # Drop the temporary columns used for calculations
    df_labels.drop(columns=['shifted', 'start', 'start_date', 'duration'], inplace=True)
    
    # Set 'start_date' as the index if needed
    df_events.set_index('start_date', inplace=True)
    df_events['season'] = df_events.index.month % 12 // 3 + 1  # This will give: 1=DJF, 2=MAM, 3=JJA, 4=SON
    # Map season numbers to season names
    season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
    df_events['season'] = df_events['season'].map(season_map)
    df_events['year'] = df_events.index.year
    dic_events[reanalysis] = df_events

# Functions

In [3]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

def plot_composites(Composites, lon, lat, quantity_name, vmin, vmax, units, colormap, \
                    pathsave=None, multiplier=1, p_values=False, p_threshold=0.05):
    """
    Plots composite maps with optional hatching for significance.

    Parameters:
    -----------
    Composites: dict
        Dictionary of composites for different weather regimes and seasons.
    lon, lat: ndarray
        Longitudes and latitudes of the data grid.
    quantity_name: str
        Name of the quantity to plot.
    vmin, vmax: float
        Minimum and maximum values for the colormap.
    units: str
        Units of the plotted quantity (for the colorbar).
    colormap: str
        Name of the colormap to use.
    pathsave: str, optional
        Path to save the figure. If None, the figure is displayed.
    multiply: float, optional
        Factor to multiply the data by before plotting.
    p_values: dict, optional
        Dictionary of p-values for each weather regime and season. Same structure as `Composites`.
    p_threshold: float, optional
        Threshold for significance (default is 0.05).
    """
    # Define the color levels and color map
    levels = np.linspace(vmin, vmax, 11)
    cmap = plt.get_cmap(colormap)

    # Create the figure and axes
    fig, axes = plt.subplots(nrows=len(Composites), ncols=5, figsize=(15, 12), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=-100)})
    axes = axes.flatten()

    season_titles = ['Winter', 'Spring', 'Summer', 'Fall', 'All year']

    # Iterate over each weather regime (rows)
    for wr in range(len(Composites)):
        # Iterate over each season (columns)
        for iseason, season in enumerate(['Winter', 'Spring', 'Summer', 'Fall', 'All year']):
            ax = axes[wr * 5 + iseason]

            # Plot the composite map
            try:
                composite = Composites[str(wr)][season][quantity_name]
            except:
                composite = Composites[wr][season][quantity_name]

            # Convert longitudes to -180 to 180 range
            lon = (lon + 180) % 360 - 180
            ax.set_extent([-80, 50, 20, 75], crs=ccrs.PlateCarree(central_longitude=-100))

            # Plot the composite values using pcolormesh
            cf = ax.pcolormesh(lon, lat, composite * multiplier, vmin=vmin, vmax=vmax, 
                               cmap=cmap, transform=ccrs.PlateCarree())

            # Add hatching based on p-values if provided
            # Add hatching based on p-values if provided
            if p_values is not None:
                try:
                    p_values_current = Composites[str(wr)][season][p_values]
                except:
                    p_values_current = Composites[wr][season][p_values]

                # Create a mask where p-values are less than the threshold
                significance_mask = (p_values_current <= p_threshold)

                # Overlay hatching on significant areas
                hatch_plot = ax.contourf(lon, lat, significance_mask, levels=[0.5, 1.5], 
                            colors='none', hatches=['.....'], transform=ccrs.PlateCarree())
                # Customize hatch properties if needed
                for collection in hatch_plot.collections:
                    collection.set_edgecolor('white')  # Change hatch color
                    collection.set_linewidth(0.)     # Change hatch line width
                # ax.contourf(lon, lat, significance_mask, levels=[0.5, 1], 
                #             colors='none', hatches=['xxx'], transform=ccrs.PlateCarree())

            # Add coastlines and gridlines
            ax.coastlines()
            ax.add_feature(cfeature.BORDERS, linestyle=':')
            ax.gridlines(draw_labels=False, linestyle='--', linewidth=0.5, alpha=0.7)

            # Ensure y-ticks are present for the first column
            if iseason == 0:
                ax.set_yticks([])  # Example y-ticks, adjust as needed
                ax.set_ylabel(f'{names_wrs[wr]}', fontsize=13)
            else:
                ax.set_yticklabels([])  # Remove y-tick labels for other columns

            # Set the title for each season column (first row only)
            if wr == 0:
                ax.set_title(season_titles[iseason], fontsize=13)

            # Set font sizes for axes labels
            ax.tick_params(axis='both', which='major', labelsize=10)

    # Add a main title to the entire plot
    fig.suptitle(f"{quantity_name} - Composites", fontsize=18, y=0.96, 
                 horizontalalignment='left', x=0.)

    # Adjust layout to prevent overlapping
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Create colorbar
    cbar_ax = fig.add_axes([0.8, 0.95, 0.2, 0.01])  # Position similar to the legend
    cbar = fig.colorbar(cf, cax=cbar_ax, orientation='horizontal')
    cbar.set_label(units, fontsize=13)
    cbar.ax.tick_params(labelsize=10)

    # Save or show the plot
    if pathsave:
        plt.savefig(pathsave, bbox_inches='tight')
        plt.close('all')
    else:
        plt.show()
        plt.close('all')

In [4]:
names_wrs = ["Pacific High","Pacific Trough","Greenland High","Atlantic High","No WR"]

seasons = ['Winter','Spring','Summer','Fall','All year']

# load quantities

In [19]:
list_names_vars = ['total_precip_da',
                # 'max_wet_spell_duration',
                # 'max_dry_spell_duration',
                'max_precip_da',
                # 'max_wet_spell_acum',
                'max_precip_5d_da',
                # 'min_precip_5d_da',
                'numberwetdays_da',
                'percentagewetdays_da']

cmaps = ['seismic_r', #totalprecip 
        # 'seismic_r', #max_wet_spell_duration
        # 'seismic', #max_dry_spell_duration
        'seismic_r', #max_precip_da
        # 'seismic_r', #max_wet_spell_acum
        'seismic_r', #max_precip_5d_da
        # 'seismic_r', #min_precip_5d_da
        'seismic_r', #numberwetdays_da
        'seismic_r'] #percentagewetdays_da]

cmaps_trends = ['BrBG', #totalprecip 
        # 'RdYlBu', #max_wet_spell_duration
        # 'RdYlBu_r', #max_dry_spell_duration
        'BrBG', #max_precip_da
        # 'RdYlBu', #max_wet_spell_acum
        'BrBG', #max_precip_5d_da
        # 'RdYlBu', #min_precip_5d_da
        'BrBG', #numberwetdays_da
        'BrBG'] #percentagewetdays_da]

amplitude_map_diffs = [100, #totalprecip
                      # 80, #max_wet_spell_duration
                      # 50, #max_dry_spell_duration
                      120, #max_precip_da 3
                      # 70, #max_wet_spell_acum 4
                      100, #max_precip_5d_da
                      # 120, #min_precip_5d_da
                      120, #numberwetdays_da 7
                      120] #percentagewetdays_da

amplitude_map_trends = [0.25*10, #totalprecip
                       # 0.025, #max_wet_spell_duration
                       # 0.06, #max_dry_spell_duration
                       0.1*10, #max_precip_da
                       # 0.5, #max_wet_spell_acum
                       0.25*10, #max_precip_5d_da
                       # 0.1, #min_precip_5d_da
                       0.05*10, #numberwetdays_da
                       0.008*1000] #percentagewetdays_da

multipliers = [10,10,10,10,1000]

units_trend = ['mm/decade', #totalprecip
              # 'days/year', #max_wet_spell_duration
              # 'days/year', #max_dry_spell_duration
              'mm/decade', #max_precip_da
              # 'mm/year', #max_wet_spell_acum
              'mm/decade', #max_precip_5d_da
              # 'mm/year', #min_precip_5d_da
              '(days/wr)/decade', #numberwetdays_da
              '%/decade',] #percentagewetdays_da

In [6]:
dic_statistics_full = {}
for name_var in list_names_vars[:3]:
    dic_temp = np.load(f'../Data_v4/TrendsPrecipitation/dic_statistics_{name_var}.npy',\
                      allow_pickle=True)[()]
    dic_statistics_full[name_var] = dic_temp

In [10]:
total_precip_da = xr.open_dataset('/glade/derecho/scratch/jhayron/TotalPrecip_WRs_v4_4wrs.nc').DailyPrecip#.compute()
lon = total_precip_da.lon.compute()
lat = total_precip_da.lat.compute()

In [20]:
for ivar in [2]:
# for ivar in [1,2,4]:
# for ivar in [2,4]:
    name_var = list_names_vars[ivar]
    print(name_var)
    dic_statistics = dic_statistics_full[name_var]
    
    # plot_composites(dic_statistics,lon,lat,\
    #                 'diff_medians',-amplitude_map_diffs[ivar],amplitude_map_diffs[ivar],\
    #                 '%',cmaps[ivar],multiplier=100, \
    #                 p_values='p_value_mannwhitneyu',
    #                 pathsave = f'../Figures/trends_precip/{name_var}_diff_medians.png')
    
    # plot_composites(dic_statistics,lon,lat,\
    #                 'diff_means',-amplitude_map_diffs[ivar],amplitude_map_diffs[ivar],\
    #                 '%',cmaps[ivar],multiplier=100, \
    #                 p_values='p_value_bootstrap',
    #                 pathsave = f'../Figures_v4/Precipitation/{name_var}_diff_means.png')
    
    plot_composites(dic_statistics,lon,lat,\
                    'trend',-amplitude_map_trends[ivar],amplitude_map_trends[ivar],\
                    units_trend[ivar],cmaps_trends[ivar],multiplier=multipliers[ivar], \
                    p_values='trend_p_value',
                    pathsave = f'../Figures_v4/Precipitation/{name_var}_trend.png')
    
    # plot_composites(dic_statistics,lon,lat,\
    #                 'sen_slope',-amplitude_map_trends[ivar],amplitude_map_trends[ivar],\
    #                 units_trend[ivar],cmaps_trends[ivar],multiplier=1, \
    #                 p_values='mann_kendall_p_value',
    #                 pathsave = f'../Figures/trends_precip/{name_var}_sentrend.png')

max_precip_5d_da


In [15]:
cmaps_trends[ivar]

'BrBG'